In [1]:
import os
import numpy as np
import pandas as pd
from keras import models
from keras import layers

Using TensorFlow backend.


In [2]:
import ml_data_stats

In [3]:
data_path = os.path.join('data', 'ml_data', 'model_data.csv')
data_df = pd.read_csv(data_path)
data_df.head()

,h_AdjTempo,h_AdjOE,h_AdjDE,h_eFG_Pct_O,h_eFG_Pct_D,h_TO_Pct_O,h_TO_Pct_D,h_OR_Pct_O,h_OR_Pct_D,h_FT_Rate_O,...,a_OppBlockPct,a_F3GRate,a_OppF3GRate,a_ARate,a_OppARate,a_StlRate,a_OppStlRate,hca,home_points,away_points
0,56.9537,96.7489,104.867,50.0378,53.1845,22.4911,19.9856,23.3696,31.5758,34.4175,...,9.6466,37.6415,35.5422,48.3266,49.7920,0.1061,0.0858,4.1,46,70
1,56.9537,96.7489,104.867,50.0378,53.1845,22.4911,19.9856,23.3696,31.5758,34.4175,...,7.4976,30.8418,31.6192,42.3701,54.7022,0.1148,0.1096,4.1,68,58
2,64.0592,110.2710,101.057,51.2293,49.3358,17.3706,19.7973,33.1795,27.7778,36.3271,...,10.6650,39.7126,44.2167,61.3793,60.6109,0.0816,0.1002,3.9,58,48
3,56.9537,96.7489,104.867,50.0378,53.1845,22.4911,19.9856,23.3696,31.5758,34.4175,...,10.5038,36.0082,39.1696,57.4040,57.1675,0.1091,0.0919,4.1,75,41
4,56.9537,96.7489,104.867,50.0378,53.1845,22.4911,19.9856,23.3696,31.5758,34.4175,...,7.0361,29.9877,34.7852,52.4793,53.1126,0.0878,0.1050,4.1,68,75


In [11]:
y = data_df[['home_points', 'away_points']]
X = data_df.drop(['home_points', 'away_points'], axis=1)
X -= ml_data_stats.mean
X = X / ml_data_stats.std
X.head()

,h_AdjTempo,h_AdjOE,h_AdjDE,h_eFG_Pct_O,h_eFG_Pct_D,h_TO_Pct_O,h_TO_Pct_D,h_OR_Pct_O,h_OR_Pct_D,h_FT_Rate_O,...,a_OppFG3Pct,a_OppFTPct,a_OppBlockPct,a_F3GRate,a_OppF3GRate,a_ARate,a_OppARate,a_StlRate,a_OppStlRate,hca
0,-2.766764,-1.10728,0.346726,0.064405,1.306807,1.583705,0.364575,-1.771111,0.319206,-0.404502,...,0.418625,0.488891,0.135216,0.510113,0.178314,-0.789246,-0.628863,0.877644,-0.509960,1.149670
1,-2.766764,-1.10728,0.346726,0.064405,1.306807,1.583705,0.364575,-1.771111,0.319206,-0.404502,...,-0.662650,-1.385344,-1.165224,-0.687194,-0.717707,-1.906266,0.359056,1.404013,1.314969,1.149670
2,-0.588638,0.71089,-0.226869,0.449733,-0.012649,-0.750542,0.283287,0.548522,-0.871364,-0.037438,...,0.010769,-0.616566,0.751488,0.874797,2.159589,1.658521,1.547871,-0.604663,0.594199,0.975749
3,-2.766764,-1.10728,0.346726,0.064405,1.306807,1.583705,0.364575,-1.771111,0.319206,-0.404502,...,-1.796955,-2.671501,0.653940,0.222517,1.006820,0.913034,0.855068,1.059150,-0.042226,1.149670
4,-2.766764,-1.10728,0.346726,0.064405,1.306807,1.583705,0.364575,-1.771111,0.319206,-0.404502,...,0.312112,0.317558,-1.444495,-0.837586,0.005414,-0.010492,0.039233,-0.229548,0.962252,1.149670


In [12]:
X.shape

(51759, 111)

In [34]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
              input_shape=(X.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(2))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [35]:
model = build_model()

In [36]:
model.fit(
    X_train,
    y_train,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
 - 2s - loss: 222.0463 - mean_absolute_error: 10.9126
Epoch 2/500
 - 2s - loss: 116.8002 - mean_absolute_error: 8.5345
Epoch 3/500
 - 2s - loss: 105.8613 - mean_absolute_error: 8.1369
Epoch 4/500
 - 2s - loss: 101.0412 - mean_absolute_error: 7.9624
Epoch 5/500
 - 2s - loss: 98.8541 - mean_absolute_error: 7.8674
Epoch 6/500
 - 2s - loss: 96.7022 - mean_absolute_error: 7.7888
Epoch 7/500
 - 2s - loss: 95.3503 - mean_absolute_error: 7.7275
Epoch 8/500
 - 2s - loss: 94.4113 - mean_absolute_error: 7.6942
Epoch 9/500
 - 2s - loss: 93.2069 - mean_absolute_error: 7.6281
Epoch 10/500
 - 2s - loss: 92.0717 - mean_absolute_error: 7.5910
Epoch 11/500
 - 2s - loss: 91.9334 - mean_absolute_error: 7.5900
Epoch 12/500
 - 2s - loss: 90.7322 - mean_absolute_error: 7.5370
Epoch 13/500
 - 2s - loss: 90.3408 - mean_absolute_error: 7.5143
Epoch 14/500
 - 2s - loss: 89.1262 - mean_absolute_error: 7.4648
Epoch 15/500
 - 2s - loss: 88.8562 - mean_absolute_error: 7.4510
Epoch 16/500
 - 2s - loss: 88

Epoch 127/500
 - 2s - loss: 55.8066 - mean_absolute_error: 5.8954
Epoch 128/500
 - 2s - loss: 55.7254 - mean_absolute_error: 5.8848
Epoch 129/500
 - 2s - loss: 55.5585 - mean_absolute_error: 5.8784
Epoch 130/500
 - 2s - loss: 55.4792 - mean_absolute_error: 5.8818
Epoch 131/500
 - 2s - loss: 55.3845 - mean_absolute_error: 5.8756
Epoch 132/500
 - 2s - loss: 55.2917 - mean_absolute_error: 5.8584
Epoch 133/500
 - 2s - loss: 55.1849 - mean_absolute_error: 5.8555
Epoch 134/500
 - 2s - loss: 54.9001 - mean_absolute_error: 5.8452
Epoch 135/500
 - 2s - loss: 54.9687 - mean_absolute_error: 5.8462
Epoch 136/500
 - 2s - loss: 54.9981 - mean_absolute_error: 5.8405
Epoch 137/500
 - 2s - loss: 54.6358 - mean_absolute_error: 5.8279
Epoch 138/500
 - 2s - loss: 54.5862 - mean_absolute_error: 5.8324
Epoch 139/500
 - 2s - loss: 54.6982 - mean_absolute_error: 5.8302
Epoch 140/500
 - 2s - loss: 54.4597 - mean_absolute_error: 5.8182
Epoch 141/500
 - 2s - loss: 54.2326 - mean_absolute_error: 5.8081
Epoch 142/

 - 2s - loss: 46.5807 - mean_absolute_error: 5.3740
Epoch 252/500
 - 2s - loss: 46.3239 - mean_absolute_error: 5.3626
Epoch 253/500
 - 2s - loss: 46.2508 - mean_absolute_error: 5.3619
Epoch 254/500
 - 2s - loss: 46.3654 - mean_absolute_error: 5.3674
Epoch 255/500
 - 2s - loss: 46.3293 - mean_absolute_error: 5.3661
Epoch 256/500
 - 2s - loss: 46.2519 - mean_absolute_error: 5.3570
Epoch 257/500
 - 2s - loss: 46.0328 - mean_absolute_error: 5.3503
Epoch 258/500
 - 2s - loss: 46.1222 - mean_absolute_error: 5.3546
Epoch 259/500
 - 2s - loss: 45.9236 - mean_absolute_error: 5.3438
Epoch 260/500
 - 2s - loss: 45.9437 - mean_absolute_error: 5.3475
Epoch 261/500
 - 2s - loss: 45.7549 - mean_absolute_error: 5.3325
Epoch 262/500
 - 2s - loss: 45.7357 - mean_absolute_error: 5.3318
Epoch 263/500
 - 2s - loss: 45.8918 - mean_absolute_error: 5.3399
Epoch 264/500
 - 2s - loss: 45.9255 - mean_absolute_error: 5.3402
Epoch 265/500
 - 2s - loss: 45.7052 - mean_absolute_error: 5.3293
Epoch 266/500
 - 2s - lo

Epoch 376/500
 - 2s - loss: 42.4130 - mean_absolute_error: 5.1275
Epoch 377/500
 - 2s - loss: 42.4470 - mean_absolute_error: 5.1345
Epoch 378/500
 - 2s - loss: 42.3804 - mean_absolute_error: 5.1357
Epoch 379/500
 - 2s - loss: 42.3929 - mean_absolute_error: 5.1234
Epoch 380/500
 - 2s - loss: 42.4107 - mean_absolute_error: 5.1309
Epoch 381/500
 - 2s - loss: 42.4303 - mean_absolute_error: 5.1243
Epoch 382/500
 - 2s - loss: 42.2512 - mean_absolute_error: 5.1184
Epoch 383/500
 - 2s - loss: 42.2959 - mean_absolute_error: 5.1191
Epoch 384/500
 - 2s - loss: 42.2997 - mean_absolute_error: 5.1169
Epoch 385/500
 - 2s - loss: 42.2112 - mean_absolute_error: 5.1109
Epoch 386/500
 - 2s - loss: 42.3044 - mean_absolute_error: 5.1246
Epoch 387/500
 - 2s - loss: 42.1204 - mean_absolute_error: 5.1110
Epoch 388/500
 - 2s - loss: 42.2668 - mean_absolute_error: 5.1211
Epoch 389/500
 - 2s - loss: 42.1686 - mean_absolute_error: 5.1074
Epoch 390/500
 - 2s - loss: 42.2327 - mean_absolute_error: 5.1181
Epoch 391/

 - 2s - loss: 40.3313 - mean_absolute_error: 4.9954


In [37]:
y_val = model.predict(X_test)

In [38]:
y_val_df = pd.DataFrame(y_val)

In [39]:
y_val_df.head()

,0,1
0,88.398872,69.243813
1,75.367729,58.599892
2,69.577637,61.833305
3,75.083519,66.449539
4,69.508163,46.780716


In [40]:
y_val_df['home_win'] = y_val_df[0] > y_val_df[1]
y_val_df.head()

,0,1,home_win
0,88.398872,69.243813,True
1,75.367729,58.599892,True
2,69.577637,61.833305,True
3,75.083519,66.449539,True
4,69.508163,46.780716,True


In [41]:
y_test['home_win'] = y_test['home_points'] > y_test['away_points']
y_test.head()

/Users/RunforrrestruN/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,home_points,away_points,home_win
14759,91,83,True
38449,73,67,True
21670,66,68,False
28606,75,69,True
19431,83,80,True


In [42]:
y_test['home_win'].mean()

0.6908809891808346

In [43]:
y_val_df['home_win'].mean()

0.8146058732612056

In [50]:
y_test = y_test.reset_index()
acc = [y_test.loc[x, 'home_win'] == y_val_df.loc[x, 'home_win'] for x in range(len(y_test))]

In [51]:
np.mean(acc)

0.7636012364760433